In [2]:
import os
os.chdir("/app")

In [3]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm
import re
import os
import time

import nbformat
import glob

import sqlite3
import pandas as pd

In [4]:
COURSE_ID = "iteration_4"

In [5]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID
# config.ExecutePreprocessor.environment = {
#     'DATA_PATH_2': '/path/to/data'
# }
config.ExecutePreprocessor.timeout = 300

In [6]:
nbg = NbGraderAPI(config=config)
gradebook = nbg.gradebook

In [7]:
ASSIGNMENT = "hw_1"
NOTEBOOK = "hse_metrics"  # without .ipynb
# MAX_SCORE = 20

MY_KERNEL = "python3"

# Collect

In [8]:
nbg.collect(ASSIGNMENT, update=False)

{'success': True,
 'log': "[INFO] Processing 81 submissions of 'hw_1' for course 'iteration_4'\n[WARNING] \n    /usr/local/share/nbgrader/exchange/iteration_4/inbound/Шевченко_Кирилл+hw_1+2025-10-11T07:19:56 UTC+00 claims to be submitted by Шевченко_Кирилл but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: Шевченко_Кирилл hw_1\n[WARNING] \n    /usr/local/share/nbgrader/exchange/iteration_4/inbound/Тищенко_Анна+hw_1+2025-10-10T15:56:52 UTC+00 claims to be submitted by Тищенко_Анна but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Submission already exists, use --update to update: Тищенко_Анна hw_1\n[WARNING] \n    /usr/local/share/nbgrader/exchange/iteration_4/inbound/Шульгин_Егор+hw_1+2025-10-10T15:56:53 UTC+00 claims to be submitted by Шульгин_Егор but is owned by dev; cheating attempt?\n    you ma

# Autograde

In [9]:
submitted_student_ids = nbg.get_submitted_students(ASSIGNMENT)
type(submitted_student_ids), len(submitted_student_ids)

(set, 81)

In [10]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 59)

In [11]:
not_autograded_student_ids = list(submitted_student_ids - autograded_student_ids)
len(not_autograded_student_ids)

22

In [21]:
student_ids_requested_to_grade = None

In [22]:
if student_ids_requested_to_grade is not None:
    student_ids_to_grade = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_grade = not_autograded_student_ids
    force = False

In [23]:
f"{force=}", student_ids_to_grade

('force=False',
 ['Nagaev_Denis',
  'Бураков_Даниил',
  'Сахаутдинова_Анжелика',
  'Дельман_Александр',
  'Лемешко_Владислав',
  'Пеганов_Никита',
  'Аскарбек_Казыбек',
  'Вольхин_Данил',
  'Дистлер_Марина',
  'Шахов_Станислав',
  'Шевченко_Кирилл',
  'Конохова_Екатерина',
  'Ширшов_Константин',
  'Козлов_Кирилл',
  'Ишмуратов_Сергей',
  'Кирильцев_Даниил',
  'Назарько_Михаил',
  'гадиев_миша',
  'Партин_Максим',
  'Холичева_Ангелина',
  'Лю_Юрий',
  'Кириенко_Владислав'])

In [24]:
results = {}
for student_id in tqdm(student_ids_to_grade):
    ts = time.time()
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=True)
    elapsed = time.time() - ts
    res["time"] = elapsed
    results[student_id] = res

  0%|          | 0/22 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Student.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Assignment.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgra

In [25]:
successes = {sid: res for sid, res in results.items() if res["success"]}
failures = {sid: res for sid, res in results.items() if not res["success"]}
len(successes), len(failures)

(22, 0)

# Get per-task scores

In [26]:
student_ids_requested_to_grade = None

In [27]:
if student_ids_requested_to_grade is not None:
    students_to_get_scores = student_ids_requested_to_grade
else:
    students_to_get_scores = nbg.get_autograded_students(ASSIGNMENT)

In [28]:
len(students_to_get_scores)

81

In [29]:
scores = {}
for student_id in tqdm(students_to_get_scores):
    nb = gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, student_id)
    student_scores = {
        "total": {"actual": nb.score, "max": nb.max_score},
        "cells": [
            {"actual": g.score, "max": g.max_score}
            for g in nb.grades
        ]
    }
    scores[student_id] = student_scores

  0%|          | 0/81 [00:00<?, ?it/s]

In [30]:
total_scores = {sid: int(s["total"]["actual"]) for sid, s in scores.items()}
len(total_scores)

81

# Make human-readable feedback

In [31]:
CELL_NAMES = [
    "precision_at_1",  
    "precision_at_5", 
    "recall_at_1", 
    "ap_at_3_for_user_2", 
    "map_at_3",
    "dcg_at_3_for_user_2",
    "idcg_at_3_for_user_2",
    "ndcg_at_3",
    "weighted_recall_correctness",
    "weighted_recall_effectiveness",
]

**Скоры предполагаются целыми**

In [32]:
feedbacks = []
for student_id, student_scores in scores.items():
    total_score = student_scores["total"]
    feedback_scores = [f"total: {total_score['actual']:.0f} / {total_score['max']:.0f}"]
    for cell_name, cell_score in zip(CELL_NAMES, student_scores["cells"]):
        feedback_scores.append(f"{cell_name}: {cell_score['actual']:.0f} / {cell_score['max']:.0f}")
    feedback = {
        "student_id": student_id,
        "feedback": ", ".join(feedback_scores),
        "score": round(total_score["actual"])
    }
    feedbacks.append(feedback)

In [33]:
results_df = pd.DataFrame(feedbacks)
print(results_df.shape)
results_df.head()

(81, 3)


,student_id,feedback,score
0,Решетняк_Анна,"total: 7 / 20, precision_at_1: 1 / 1, precisio...",7
1,Шестаков_Денис,"total: 8 / 20, precision_at_1: 1 / 1, precisio...",8
2,Абрамов_Игорь,"total: 6 / 20, precision_at_1: 1 / 1, precisio...",6
3,Бутаков_Максим,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
4,Бутусова_Анастасия,"total: 14 / 20, precision_at_1: 1 / 1, precisi...",14


In [34]:
filename_prefix = f"{ASSIGNMENT}_results_part_"
filename_pattern = f"{filename_prefix}*.csv"
existing_files = glob.glob(filename_pattern)
existing_student_ids = set()
for fn in existing_files:
    df = pd.read_csv(fn)
    existing_student_ids.update(df["student_id"].tolist())
len(existing_student_ids)

59

In [35]:
results_df = results_df.loc[~results_df["student_id"].isin(existing_student_ids)]
len(results_df)

22

In [36]:
existing_parts = [int(fn.removeprefix(filename_prefix).removesuffix(".csv")) for fn in existing_files]
if len(existing_parts) > 0:
    next_part = max(existing_parts) + 1
else:
    next_part = 1
results_file_name = f"{filename_prefix}{next_part}.csv"
results_file_name

'hw_1_results_part_2.csv'

In [38]:
results_df.sort_values("student_id", inplace=True)
results_df

/tmp/ipykernel_93/1864736668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df.sort_values("student_id", inplace=True)


,student_id,feedback,score
22,Nagaev_Denis,"total: 6 / 20, precision_at_1: 1 / 1, precisio...",6
47,Аскарбек_Казыбек,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
43,Бураков_Даниил,"total: 18 / 20, precision_at_1: 1 / 1, precisi...",18
27,Вольхин_Данил,"total: 8 / 20, precision_at_1: 1 / 1, precisio...",8
62,Дельман_Александр,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
64,Дистлер_Марина,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
13,Ишмуратов_Сергей,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20
60,Кириенко_Владислав,"total: 19 / 20, precision_at_1: 1 / 1, precisi...",19
58,Кирильцев_Даниил,"total: 6 / 20, precision_at_1: 1 / 1, precisio...",6
69,Козлов_Кирилл,"total: 20 / 20, precision_at_1: 1 / 1, precisi...",20


In [39]:
results_df.to_csv(results_file_name)